# Quantum Teleporation -- 2 Node

In this demo, we'll demonstrate the simulation of the teleportation application. The network topology has two nodes: Alice and Bob. Alice will teleport a qubit to Bob. 

## 1. The Basics

The figure below shows the quantum circuit of teleportation, as well as showing which qubits belong to Alice and which qubit belongs to Bob. We need to turn this quantum circuit into a network protocol.

<img src="./figures/teleport-circuit.png" width="600">

**Set up**: Assume node Alice and Bob both have two quantum memories, one labeled as data memory and the other labeled as communication memory. There is a quantum channel between Alice and Bob, and the BSM node is in the middle of the quantum channel.

Let's break down quantum teleportation into 4 steps.

**Step 1**: The tradition is to let Alice send a qubit to Bob. So, Alice prepares $\ket{\psi}$ at memory $d_1$.

<img src="./figures/teleport-1.png" width="600">

**Step 2**: Generate an entangled pair between Alice's memory $c_1$ and Bob's memory $c_2$.

<img src="./figures/teleport-2.png" width="600">

**Step 3**: Alice does a Bell measurement on it's memory $d_1$ and $c_1$. The measurement results are sent to Bob.

<img src="./figures/teleport-3.png" width="600">

**Step 4**: Bob does the correction on $c_2$ upon receiving the measurement results. Then $\ket{\psi}$ is teleported to Bob's memory $c_2$.

<img src="./figures/teleport-4.png" width="600">

## 2. The Code

**The imports**

- `log`: defines the behavior for the SeQUeNCe logging system
- `TeleportApp`: TeleportApp is a specialized RequestApp that implements quantum teleportation. It handles the teleportation protocol between two quantum nodes (Alice and Bob).
- `DQCNode`: Node that supports Distributed Quantum Computing
- `DQCNetTopo`: Class for generating a distributed quantum computing network with distributed quantum computing nodes
- `random_state`: Function to generate a random quantum state
- `verify_same_state_vector`: Function to verify whether two state vectors are the same or not


In [1]:
import numpy as np
import sequence.utils.log as log
from sequence.app.teleport_app import TeleportApp
from sequence.topology.node import DQCNode
from sequence.topology.dqc_net_topo import DQCNetTopo
from sequence.kernel.quantum_utils import random_state, verify_same_state_vector
from sequence.constants import MILLISECOND

### 2.1 The teleport() function

Teleport a $\ket{\psi}$ from Alice to Bob

In [2]:
def teleport(psi: np.array, seed: int = 0) -> np.array:
    """Do a teleportation
    
    Args:
        psi (np.array): the state to be teleported at Alice
        seed (int): the random seed
    Return:
        np.array: the state received at Bob
    """
    # read the json config file, initialize the topology
    topo = DQCNetTopo("teleport_2node.json")
    tl   = topo.tl  # timeline

    # set up log
    log_filename = 'teleport_2node.log'
    log.set_logger(__name__, tl, log_filename)
    log.set_logger_level('INFO')
    modules = ['generation', 'teleportation', 'teleport_app']
    for module in modules:
        log.track_module(module)

    # get alice and bob node
    nodes = topo.nodes[DQCNetTopo.DQC_NODE]
    alice = next(n for n in nodes if n.name=="alice")
    bob   = next(n for n in nodes if n.name=="bob")

    # 0) set seed for the bsm node
    bsm_nodes = topo.nodes.get(DQCNetTopo.BSM_NODE, [])
    for bsm_node in bsm_nodes:
        bsm_node.set_seed(seed)
    
    # 1) Prepare |ψ> in Alice’s data memory
    data_memo_arr = alice.get_component_by_name(alice.data_memo_arr_name)
    data_memo_arr[0].update_state(psi)

    # 2) Attach the TeleportApp on both nodes
    alice_app = TeleportApp(alice)
    bob_app   = TeleportApp(bob)

    # 3) Kick off teleport
    alice_app.start(
        responder   = bob.name,
        start_t     = 1  * MILLISECOND,
        end_t       = 100 * MILLISECOND,
        memory_size = 1,
        fidelity    = 0.8,
        data_memory_index = 0
    )
    
    # 4) Run the simulation
    tl.init()
    tl.run()

    # 5) simulation finished, read out Bob’s communication qubit state
    timestamp, teleported_qubit = bob_app.results[0]

    return np.array(teleported_qubit)

### 2.2 The main() function

In [3]:
def main():

    psi = np.array(random_state())
    seed = np.random.randint(0, 1000)
    
    teleported_psi = teleport(psi, seed)  # run the teleporation
    
    print(f'psi            = {psi}')
    print(f'teleported psi = {teleported_psi}')
    
    assert verify_same_state_vector(teleported_psi, psi), f"teleported state {teleported_psi} != original {psi}"
    
    print('\npsi = teleported_psi --> teleporation success!')
    
    print('\nThe teleport_2node.log:\n')
    with open("teleport_2node.log", "r") as f:
        logs = f.read()
        print(logs)

In [38]:
# please run this cell repeatedly
main()

psi            = [ 0.55050402+0.j         -0.7700823 -0.32236404j]
teleported psi = [ 0.55050402+0.j         -0.7700823 -0.32236404j]

psi = teleported_psi --> teleporation success!

The teleport_2node.log:

3,002,500,010        INFO    generation             alice failed entanglement of memory alice.MemoryArray[0]
3,002,500,010        INFO    generation             bob failed entanglement of memory bob.MemoryArray[0]
5,005,012,510        INFO    generation             alice failed entanglement of memory alice.MemoryArray[0]
5,005,012,510        INFO    generation             bob failed entanglement of memory bob.MemoryArray[0]
8,510,037,510        INFO    generation             alice failed entanglement of memory alice.MemoryArray[0]
8,510,037,510        INFO    generation             bob failed entanglement of memory bob.MemoryArray[0]
10,512,550,010       INFO    generation             alice failed entanglement of memory alice.MemoryArray[0]
10,512,550,010       INFO    generation  

# 3 Analyze the log

<img src="./figures/teleport-timeline.png" width="800">